Importar bibliotecas

In [19]:
import pandas as pd
import numpy as np
import html
import nltk
import regex as re
import string
import spacy
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [20]:
# nltk.download()

In [21]:
#!pip install spacy
#!python -m spacy download pt

Importar dados

In [22]:
data = pd.read_csv('../../data/raw/bncc_first_classifier.csv')

In [23]:
data_clone = data.copy()

In [24]:
# renomeando as colunas do dataset

data_clone.columns = ['id', 'questoes', 'tipoQuestoes', 'topico', 'slug', 'materia', 'slug.1','etapaEnsino']
data_clone.head()

,id,questoes,tipoQuestoes,topico,slug,materia,slug.1,etapaEnsino
0,3486670,<p>Com base em seus conhecimentos sobre o comp...,2,História da Arte,historia-da-arte,Arte,arte,Fundamental II
1,3343927,"<p>&ldquo;Achar um n&uacute;mero que, somado c...",2,Álgebra: Equações do 2º grau: Equação do 2º gr...,algebra-equacoes-do-2o-grau-equacao-do-2o-grau...,Matemática,matematica,Fundamental II
2,1927600,<p>2. Complete os espaços abaixo com os artigo...,4,Outros,outros,Espanhol,espanhol,Fundamental II
3,106855,<p><div> A respeito dos serviços públicos e da...,2,Responsabilidade Civil do Estado,responsabilidade-civil-do-estado,Direito Administrativo,direito-administrativo,Concurso
4,3691951,"<p style=""padding:0px;margin:10px 0px 0px;outl...",2,Outros,outros,Química,quimica,Fundamental II


## Pre-processamento

* Escrevendo funcões para etapa de pre-processamento de Texto usando regex 

In [40]:
# Função para remover tags de html usando regex

def remove_tags(text: str) -> str:
    pattern = re.compile("<.*?>")
    cleantext = re.sub(pattern, " ", text).replace(u"\xa0", u" ")
    return cleantext

# Função para deixar todo o texto em caixa baixa
def to_lower(text: str) -> str:
    return text.lower()

# Função para remover numeros
def remove_numbers(text: str) -> str:
    pattern = re.compile("[0-9]+")
    clean_text = re.sub(pattern, " ", text)
    return clean_text

# Função para remover stopwords em portugues; preposições, conjunções, artigos, etc
def remove_standard_stopwords_pt(text: str) -> str:
    try:
        stop_words = set(stopwords.words("portuguese"))
        word_tokens = word_tokenize(text)
        filtered_sentence = [w for w in word_tokens if not w in stop_words]
        final_sentence = " ".join(filtered_sentence)
        return final_sentence
    except:
        return text

def remove_standard_stopwords_en(text: str) -> str:
    try:
        stop_words = set(stopwords.words("english"))
        word_tokens = word_tokenize(text)
        filtered_sentence = [w for w in word_tokens if not w in stop_words]
        final_sentence = " ".join(filtered_sentence)
        return final_sentence
    except:
        return text

# Função para remover caracteres especiais e pontuação
def remove_punctuation(text: str) -> str:
    pattern = re.compile("[\d\.\,\*]")
    clean_text = re.sub(pattern, " ", text)
    return clean_text

# Função para remover aspas 
def remove_quotes(text: str) -> str:
    pattern = re.compile(r'“')
    clean_text = re.sub(pattern, " ", text)
    return clean_text

In [38]:
# resolve problema de codificação html da coluna 'questoes'
data_clone['questoes_html']= data['question'].astype('str').apply(html.unescape)

# limpando o texto utilizando as funçoes criadas
data_clone['questoes_tags'] = data_clone['questoes_html'].apply(remove_tags)
data_clone['questoes_lower'] = data_clone['questoes_tags'].apply(to_lower)
data_clone['questoes_numbers'] = data_clone['questoes_lower'].apply(remove_numbers)
data_clone['questoes_stopw_pt'] = data_clone['questoes_numbers'].apply(remove_standard_stopwords_pt)
data_clone['questoes_stopw_en'] = data_clone['questoes_stopw_pt'].apply(remove_standard_stopwords_en)
data_clone['questoes_punct'] = data_clone['questoes_stopw_en'].apply(remove_punctuation)
# data_clone['questoes_quotes'] = data_clone['questoes_punct'].apply(remove_quotes)

In [41]:
data_clone['questoes_quotes'] = data_clone['questoes_punct'].apply(remove_quotes)
data_clone.head()

,id,questoes,tipoQuestoes,topico,slug,materia,slug.1,etapaEnsino,questoes_html,questoes_tags,questoes_lower,questoes_numbers,questoes_stopw_pt,questoes_stopw_en,questoes_punct,questoes_quotes
0,3486670,<p>Com base em seus conhecimentos sobre o comp...,2,História da Arte,historia-da-arte,Arte,arte,Fundamental II,<p>Com base em seus conhecimentos sobre o comp...,Com base em seus conhecimentos sobre o compos...,com base em seus conhecimentos sobre o compos...,com base em seus conhecimentos sobre o compos...,base conhecimentos sobre compositor john cage ...,base conhecimentos sobre compositor john cage ...,base conhecimentos sobre compositor john cage ...,base conhecimentos sobre compositor john cage ...
1,3343927,"<p>&ldquo;Achar um n&uacute;mero que, somado c...",2,Álgebra: Equações do 2º grau: Equação do 2º gr...,algebra-equacoes-do-2o-grau-equacao-do-2o-grau...,Matemática,matematica,Fundamental II,"<p>“Achar um número que, somado com 2 seja igu...","“Achar um número que, somado com 2 seja igual...","“achar um número que, somado com 2 seja igual...","“achar um número que, somado com seja igual...","“ achar número , somado igual inverso ” . equa...","“ achar número , somado igual inverso ” . equa...",“ achar número somado igual inverso ” equa...,achar número somado igual inverso ” equa...
2,1927600,<p>2. Complete os espaços abaixo com os artigo...,4,Outros,outros,Espanhol,espanhol,Fundamental II,<p>2. Complete os espaços abaixo com os artigo...,2. Complete os espaços abaixo com os artigos ...,2. complete os espaços abaixo com os artigos ...,. complete os espaços abaixo com os artigos ...,. complete os espaços abaixo com os artigos ...,. complete os espaços abaixo com os artigos ...,complete os espaços abaixo com os artigos ...,complete os espaços abaixo com os artigos ...
3,106855,<p><div> A respeito dos serviços públicos e da...,2,Responsabilidade Civil do Estado,responsabilidade-civil-do-estado,Direito Administrativo,direito-administrativo,Concurso,<p><div> A respeito dos serviços públicos e da...,A respeito dos serviços públicos e da respo...,a respeito dos serviços públicos e da respo...,a respeito dos serviços públicos e da respo...,respeito serviços públicos responsabilidade ci...,respeito serviços públicos responsabilidade ci...,respeito serviços públicos responsabilidade ci...,respeito serviços públicos responsabilidade ci...
4,3691951,"<p style=""padding:0px;margin:10px 0px 0px;outl...",2,Outros,outros,Química,quimica,Fundamental II,"<p style=""padding:0px;margin:10px 0px 0px;outl...",A chuva ácida é um fenômeno provocado pela po...,a chuva ácida é um fenômeno provocado pela po...,a chuva ácida é um fenômeno provocado pela po...,chuva ácida fenômeno provocado poluição atmosf...,chuva ácida fenômeno provocado poluição atmosf...,chuva ácida fenômeno provocado poluição atmosf...,chuva ácida fenômeno provocado poluição atmosf...


Verifica a existência de questões para concursos e outros tipos que não é interessante para análise

In [ ]:
data_clone['etapaEnsino'].value_counts()

Selecionando apenas questões relacionadas ao Ensino Fundamental e Médio

In [ ]:
data_step_clean = data_clone[data_clone["etapaEnsino"].isin(["Médio & Pré-Vestibular", "Fundamental II", "Fundamental I"])]
data_step_clean.head()

A variável tipoQuestoes contém items dos tipos:

1. Somatória
2. Múltipla Escolha
3. Certo e Errado
4. Discursiva
5. Redação ou items que é necessário enviar foto da resolução
6. Questões com mais de uma alternativa certa
7. Não representa nada em específico

Iremos remover questoes tipo 5 e 6 pois não se adequam ao tipo de alvo do nosso problema.

In [ ]:
data_step_clean["tipoQuestoes"].value_counts(ascending=True)

Obs.: Questoes tipo Multipla escolha e Discursiva somam a maioria das questões

Remover as questoes tipo 5 e 6

In [ ]:
data_step_clean_ques = data_step_clean[~data_step_clean["tipoQuestoes"].isin([5, 6])]

In [ ]:
data_step_clean["materia"].value_counts(ascending=True)

Selecionando apenas as matétias listadas na BNCC para Ensino médio e Fundamental 1 e 2

Tokenizaçao das Questoes usando NLTK

In [ ]:
data_step_clean_ques['questoes_tokens'] = data_step_clean_ques['questoes_punct'].apply(word_tokenize)
data_step_clean_ques.head()